# Model Analysis

In [1]:
import numpy as np
import os
os.chdir('genre_classification_289a/src')
from analysis_utils import load_available_f1s, load_available_loss_histories

In [2]:
# set possible file attrs
dataset_name = "fma_medium"
possible_targets = ["subgenres", "mfcc", "genre"]
possible_layers = [7,6,5,4]

## Table of F1 (micro) Performance

In [3]:
mlp_f1s = load_available_f1s(dataset_name, possible_targets, possible_layers)

for f1 in mlp_f1s:
    print(f1)

{'targets': ['subgenres'], 'layer': 7, 'f1': array([0.86016313])}
{'targets': ['subgenres', 'mfcc', 'genre'], 'layer': 7, 'f1': array([0.97473573])}
{'targets': ['subgenres', 'mfcc', 'genre'], 'layer': 6, 'f1': array([0.96113189])}
{'targets': ['subgenres', 'mfcc', 'genre'], 'layer': 5, 'f1': array([0.79049498])}
{'targets': ['subgenres', 'mfcc', 'genre'], 'layer': 4, 'f1': array([0.67209446])}


## Plots of Losses

In [ ]:
mlp_losses_histories = load_available_loss_histories(dataset_name, possible_targets, possible_layers)

for mlp_loss_history in mlp_loss_histories:
    print(mlp_loss_history)

NameError: name 'mlp_loss_histories' is not defined